In [1]:
from core.generate_streets import read_overture_region_streets

import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

## Run sequentially for all streets

In [2]:
streets_dir = '/data/uscuni-ulce/overture_streets/'
regions_datadir = "/data/uscuni-ulce/"

In [3]:
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_hulls.shape

(847, 1)

In [4]:
# %%capture cap

# for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows():\

#     if region_id != 69333: continue
#     print('Processing', region_id, datetime.datetime.now())
#     streets = download_overture_streets(region_hull.iloc[0], region_id)
#     streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

## Run in parallel for all non-procesed streets in region_hulls

In [5]:
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [8]:
downloaded_streets = glob.glob(streets_dir + '*.pq')
downloaded_streets = [int(s.split('_')[-1][:-3]) for s in downloaded_streets]

In [9]:
region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets), ]
region_hulls

,convex_hull
labels,
139621,"POLYGON ((5066400 3660200, 5062200 3660500, 50..."
139674,"POLYGON ((5019000 3690300, 5011800 3693100, 50..."
139764,"POLYGON ((5051200 3623600, 5015000 3627900, 50..."
140420,"POLYGON ((5072300 3665600, 5070100 3666400, 50..."
141272,"POLYGON ((5062200 3733800, 5057700 3735500, 50..."
141366,"POLYGON ((5090900 3674900, 5087900 3675100, 50..."
142100,"POLYGON ((5137100 3607000, 5087800 3613300, 50..."
143701,"POLYGON ((5150200 3661900, 5138200 3666400, 51..."
145906,"POLYGON ((5186600 3540100, 5180500 3541300, 51..."


In [10]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id) for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)


Processing 142100 2024-11-29 17:53:07.918495
Processing 139621 2024-11-29 17:53:07.742506
Processing 145906 2024-11-29 17:53:07.886200
Processing 141272 2024-11-29 17:53:07.776731
Processing 143701 2024-11-29 17:53:07.851934
Processing 147112 2024-11-29 17:53:07.862589
Processing 149997 2024-11-29 17:53:07.876798
Processing 152511 2024-11-29 17:53:07.889590
Processing 150044 2024-11-29 17:53:07.861481
Processing 152081 2024-11-29 17:53:07.889544
Processing 148085 2024-11-29 17:53:07.758667
Processing 141366 2024-11-29 17:53:07.738276
Processing 139764 2024-11-29 17:53:07.736460
Processing 148018 2024-11-29 17:53:07.872263
Processing 147634 2024-11-29 17:53:07.874682
Processing 151676 2024-11-29 17:53:07.961890
Processing 146285 2024-11-29 17:53:07.681082
Processing 139674 2024-11-29 17:53:07.868017
Processing 140420 2024-11-29 17:53:07.649524


In [29]:
gdf = gpd.read_parquet(streets_dir + f'streets_{69333}.pq')

In [32]:
# gdf.explore()

1

In [21]:
cap.stderr

'/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.\n  warnings.warn(\n'

In [22]:
gdf = gpd.read_parquet(streets_dir + f'streets_{69333}.pq')

In [ ]:
for region_id, region_hull in region_hulls.iterrows():
    streets.append(streets.to_parquet(streets_dir + f'streets_{region_id}.pq'))